In [1]:
import numpy as np
import sys
import dimod
import matplotlib.pyplot as plt

from dwave.system import EmbeddingComposite, DWaveSampler
from pythia_generator import generator
from qubo_generator import make_qubo
from embedding_generator import generate_embedding

In [2]:
# Generate a Pythia event
particles = generator()

In [3]:
# Use particles in event to generate qubo
qubo, offset = make_qubo(particles, np.pi/4.)

In [18]:
# Submit qubo to D-Wave

sampler = EmbeddingComposite(DWaveSampler(solver='Advantage_system1.1'))
response = sampler.sample_qubo(qubo, num_reads=10000,return_embedding=True)

In [19]:
qpu_energies = []
qpu_chb_fraction = []

for datum in response.data(['sample', 'energy', 'chain_break_fraction']):
    qpu_energies.append(datum.energy)
    qpu_chb_fraction.append(datum.chain_break_fraction)

    #print(datum.sample, datum.energy, datum.chain_break_fraction)

In [24]:
response_adj = sampler.sample_qubo(qubo, num_reads=10000,return_embedding=True, chain_strength = 3000)

In [25]:
qpu_energies_adj = []
qpu_chb_fraction_adj = []

for datum in response_adj.data(['sample', 'energy', 'chain_break_fraction']):
    qpu_energies_adj.append(datum.energy)
    qpu_chb_fraction_adj.append(datum.chain_break_fraction)

In [37]:
from dwave.system import VirtualGraphComposite

qpu = DWaveSampler(solver='Advantage_system1.1')

In [38]:
qpu.properties['extended_j_range']

[-2.0, 1.0]

In [42]:
embedding = generate_embedding(len(particles))

virt_sampler = VirtualGraphComposite(qpu, embedding, chain_strength=1) #doctest: +SKIP

Embedding on 595 spin variables


In [43]:
sampleset = virt_sampler.sample_qubo(qubo, num_reads=10)

BinaryQuadraticModelStructureError: given bqm does not match the sampler's structure